In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import sys
import os
# sys.path.append(os.path.abspath('../..'))  # 添加项目根目录到Python路径


from dotenv import load_dotenv

load_dotenv()


False

In [2]:
sys.path.append(os.path.abspath('../..')) 

In [3]:
from llama_index.llms.litellm import LiteLLM
from llama_index.core import Settings
import os
from llama_index.core.llms import ChatMessage

C:\Users\admin\anaconda3\envs\python_class_310\lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [4]:
from llama_index.llms.deepseek import DeepSeek

# you can also set DEEPSEEK_API_KEY in your environment variables
llm = DeepSeek(model="deepseek-chat", api_key="sk-7fe9a5edf1f64ff5a748b2509e4fb34d")

# You might also want to set deepseek as your default llm
# from llama_index.core import Settings
# Settings.llm = llm

In [5]:
from llama_index.core.tools import FunctionTool
from research_agent.core.mysql_utils import MySQL
from research_agent.core.generate_patent_chart import Patent_Chart_Generator
from research_agent.core.web_search import WebSearch

my_sql = MySQL()
chart_generator = Patent_Chart_Generator()
web_searcher = WebSearch()

In [6]:
Settings.llm = llm

In [7]:
#input_patent_into_mysql_tool = FunctionTool.from_defaults(fn=my_sql.create_table_and_import_excel_to_mysql)
query_patent_from_mysql_tool = FunctionTool.from_defaults(fn=my_sql.execute_query_to_markdown)
get_top_n_countries_by_patent_count_tool = FunctionTool.from_defaults(fn=my_sql.get_top_n_countries_by_patent_count)
get_top_n_applicants_by_patent_count_tool = FunctionTool.from_defaults(fn=my_sql.get_top_n_applicants_by_patent_count)
get_top_ipc_yearly_patent_counts_tool = FunctionTool.from_defaults(fn=my_sql.get_top_ipc_yearly_patent_counts)
get_ipc_description_tool = FunctionTool.from_defaults(fn=chart_generator.get_ipc_description)
web_search_tool = FunctionTool.from_defaults(fn=web_searcher.batch_async_queries)

In [8]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)
from llama_index.core.agent import AgentRunner

worker = FunctionCallingAgentWorker.from_tools(
    [get_top_ipc_yearly_patent_counts_tool,get_ipc_description_tool,query_patent_from_mysql_tool,web_search_tool,get_top_n_countries_by_patent_count_tool,get_top_n_applicants_by_patent_count_tool], 
    verbose=True
)
agent = AgentRunner(
    worker
)


        "输出整体专利的技术布局的年度变化情况",\
        "先获取专利申请量排名前5的当前申请(专利权)人国家，然后输出每个国家的技术布局的年度变化情况"
    注意，请以markdown格式输出上述数据，在输出每类数据之前，添加说明，说明这个数据是什么
    不要输出任何其他信息

In [9]:
response = agent.chat(
    r'''我现在有个实验动物领域的专利数据集，放在patent_data数据表中，
    这批专利的字段包括['序号', '公开(公告)号', '标题', '发明人', '当前申请(专利权)人', '标题(译)(简体中文)', '[标]当前申请(专利权)人', '受理局', '申请年', '公开(公告)年', '授权年', 'IPC主分类号', 'IPC分类号', '法律状态/事件', '简单法律状态', '当前申请(专利权)人国家', '原始申请(专利权)人国家', '[标]技术功效短语', '技术领域', '第一发明人', '[标]当前申请(专利权)人类型', '简单同族编号', '简单同族专利申请数量']\
    请获取专利申请量排名前10的当前申请(专利权)人,并分别返回这些申请人的技术布局的年度变化情况
    注意，请以markdown格式输出每个申请人的数据，在输出数据之前，添加说明，解释这个数据是什么
    ''')


Added user message to memory: 我现在有个实验动物领域的专利数据集，放在patent_data数据表中，
    这批专利的字段包括['序号', '公开(公告)号', '标题', '发明人', '当前申请(专利权)人', '标题(译)(简体中文)', '[标]当前申请(专利权)人', '受理局', '申请年', '公开(公告)年', '授权年', 'IPC主分类号', 'IPC分类号', '法律状态/事件', '简单法律状态', '当前申请(专利权)人国家', '原始申请(专利权)人国家', '[标]技术功效短语', '技术领域', '第一发明人', '[标]当前申请(专利权)人类型', '简单同族编号', '简单同族专利申请数量']\
    请获取专利申请量排名前10的当前申请(专利权)人,并分别返回这些申请人的技术布局的年度变化情况
    注意，请以markdown格式输出每个申请人的数据，在输出数据之前，添加说明，解释这个数据是什么
    
=== Calling Function ===
Calling function: get_top_n_applicants_by_patent_count with args: {"table_name": "patent_data", "top_n": 10}
=== Function Output ===
['天津开发区合普工贸有限公司', '中国人民解放军空军军医大学', '中国科学院深圳先进技术研究院', '四川大学华西医院', '昆明医科大学', '中国人民解放军军事科学院军事医学研究院', '复旦大学附属中山医院', '四川大学', '江汉大学', '吉林大学']
=== Calling Function ===
Calling function: get_top_ipc_yearly_patent_counts with args: {"applicant": "\u5929\u6d25\u5f00\u53d1\u533a\u5408\u666e\u5de5\u8d38\u6709\u9650\u516c\u53f8", "table_name": "patent_data"}
=== Function Output ===
| 申请年 | 向动物体内或体上引入固体、液体

In [16]:
# 利用检索引擎获取上述数据的相关背景知识
# 
# 将上述背景知识和上述数据相结合，撰写专利申请趋势分析
response_1 = agent.chat(
    f'''我现在有个实验动物领域的专利统计数据如下:
    {response.response}
    为撰写专利申请趋势分析使用，需要在互联网上检索一下与各个申请人和技术研发相关背景信息，请以合法json格式输出需要检索的内容，输出形式为:{{'answer':['检索内容1','检索内容2',...]}},
    不要输出任何其他内容
    ''')


Added user message to memory: 我现在有个实验动物领域的专利统计数据如下:
    以下是根据实验动物领域的专利数据集（`patent_data`表）分析的结果，展示了专利申请量排名前10的当前申请(专利权)人及其技术布局的年度变化情况。每个表格显示了该申请人在不同年份中排名前十的IPC技术类别的专利数量分布。

---

### 1. 天津开发区合普工贸有限公司
**说明**：该表格展示了天津开发区合普工贸有限公司在不同年份中排名前十的IPC技术类别的专利数量分布。

| 申请年 | 向动物体内或体上引入固体、液体或气体药物或其他物质的器械或方法（用于繁殖或授精的入A61D 19/00） 〔5〕 | 用不包括在G01N 1/00至G01N 31/00组中的特殊方法来研究或分析材料  | 在A61B 1/00至A61B 18/00各组中都不包含的手术或诊断用的仪器、器械或附件，例如立体定位术、消毒作业、脱位处理、伤口边缘防护器（保护面罩入A41D 13/11；外科医生或患者用衣或服装入A41D 13/12，取得、处理或贮存体液的装置入A61M 1/00） |
| ------ | ------ | ------ | ------ |
| 2011 | 4 | 2 | 0 |
| 2013 | 2 | 0 | 1 |
| 2017 | 2 | 0 | 0 |
| 2020 | 4 | 0 | 0 |

---

### 2. 中国人民解放军空军军医大学
**说明**：该表格展示了中国人民解放军空军军医大学在不同年份中排名前十的IPC技术类别的专利数量分布。

| 申请年 | 向动物体内或体上引入固体、液体或气体药物或其他物质的器械或方法（用于繁殖或授精的入A61D 19/00） 〔5〕 | 为手术目的支撑或束缚动物的器械 | 兽医用外科器械 | 动物的房舍；所用设备（建筑结构，建筑特点入E04；建筑通风入F24F） | 压力输注，例如用泵 〔5〕 |
| ------ | ------ | ------ | ------ | ------ | ------ |
| 2021 | 2 | 0 | 0 | 0 | 1 |
| 2022 | 2 | 0 | 0 | 1 | 0 |
| 2023 | 2 | 0 | 1 | 0 | 0 |
| 20

In [18]:
response_1.response

"{'answer':['天津开发区合普工贸有限公司 实验动物领域 技术研发背景','中国人民解放军空军军医大学 实验动物领域 技术研发背景','中国科学院深圳先进技术研究院 实验动物领域 技术研发背景','四川大学华西医院 实验动物领域 技术研发背景','昆明医科大学 实验动物领域 技术研发背景','中国人民解放军军事科学院军事医学研究院 实验动物领域 技术研发背景','复旦大学附属中山医院 实验动物领域 技术研发背景','四川大学 实验动物领域 技术研发背景','江汉大学 实验动物领域 技术研发背景','吉林大学 实验动物领域 技术研发背景']}"

In [19]:
import json_repair

In [21]:
queries = json_repair.loads(response_1.response)['answer']

In [25]:
web_search_result = await web_searcher.batch_async_queries(queries)

In [29]:
query_result = [{'query':q,'result':r} for q,r in zip(queries,web_search_result)]

In [25]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 5ae7f175-c704-4fa8-9035-555f77d822ce: 1
Table 'patent_data' created and data from 'C:\Users\admin\patent_survey_draft\20250320012345555.XLSX' has been successfully imported.


In [28]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 5ae7f175-c704-4fa8-9035-555f77d822ce: 1


TaskStep(task_id='5ae7f175-c704-4fa8-9035-555f77d822ce', step_id='4c23b812-4dda-426a-bf12-7f80e80c679e', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [29]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

True


In [30]:
step_output

TaskStepOutput(output=AgentChatResponse(response='None', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None), task_step=TaskStep(task_id='5ae7f175-c704-4fa8-9035-555f77d822ce', step_id='4c23b812-4dda-426a-bf12-7f80e80c679e', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[], is_last=True)

In [31]:
response = agent.finalize_response(task.task_id)

In [32]:
print(str(response))

None


In [10]:
response = agent.chat(
    r'''我现在有个实验动物领域的专利数据集，放在"C:\Users\admin\patent_survey_draft\20250320012345555.XLSX"中\
    这批专利的字段包括['序号', '公开(公告)号', '标题', '发明人', '当前申请(专利权)人', '标题(译)(简体中文)', '[标]当前申请(专利权)人', '受理局', '申请年', '公开(公告)年', '授权年', 'IPC主分类号', 'IPC分类号', '法律状态/事件', '简单法律状态', '当前申请(专利权)人国家', '原始申请(专利权)人国家', '[标]技术功效短语', '技术领域', '第一发明人', '[标]当前申请(专利权)人类型', '简单同族编号', '简单同族专利申请数量']\
    请将此数据集存放到数据库中，并从中获取相关专利数据，以回答下面问题:
        "专利申请量排名前N的申请人（当前申请(专利权)人）及其技术布局的年度变化情况如何？",\
        请注意,创建数据库查询语句时候，数据表字段名就是上述专利的字段名;申请人某一年份的技术布局需要看这个申请人在该年度申请专利的IPC分类号分布情况，并通过获取每个IPC分类号
        的含义，得到该申请人的技术布局情况''')

Added user message to memory: 我现在有个实验动物领域的专利数据集，放在"C:\Users\admin\patent_survey_draft\20250320012345555.XLSX"中\
    这批专利的字段包括['序号', '公开(公告)号', '标题', '发明人', '当前申请(专利权)人', '标题(译)(简体中文)', '[标]当前申请(专利权)人', '受理局', '申请年', '公开(公告)年', '授权年', 'IPC主分类号', 'IPC分类号', '法律状态/事件', '简单法律状态', '当前申请(专利权)人国家', '原始申请(专利权)人国家', '[标]技术功效短语', '技术领域', '第一发明人', '[标]当前申请(专利权)人类型', '简单同族编号', '简单同族专利申请数量']\
    请将此数据集存放到数据库中，并从中获取相关专利数据，以回答下面问题:
        "专利申请量排名前N的申请人（当前申请(专利权)人）及其技术布局的年度变化情况如何？",\
        请注意,创建数据库查询语句时候，数据表字段名就是上述专利的字段名;申请人某一年份的技术布局需要看这个申请人在该年度申请专利的IPC分类号分布情况，并通过获取每个IPC分类号
        的含义，得到该申请人的技术布局情况
=== Calling Function ===
Calling function: create_table_and_import_excel_to_mysql with args: {"excel_file": "C:\\Users\\admin\\patent_survey_draft\\20250320012345555.XLSX", "table_name": "patent_data"}


C:\Users\admin\anaconda3\envs\python_class_310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


=== Function Output ===
Table 'patent_data' created and data from 'C:\Users\admin\patent_survey_draft\20250320012345555.XLSX' has been successfully imported.
=== Calling Function ===
Calling function: execute_query_to_markdown with args: {"sql": "SELECT `\u5f53\u524d\u7533\u8bf7(\u4e13\u5229\u6743)\u4eba`, `\u7533\u8bf7\u5e74`, `IPC\u4e3b\u5206\u7c7b\u53f7` FROM patent_data ORDER BY `\u5f53\u524d\u7533\u8bf7(\u4e13\u5229\u6743)\u4eba`, `\u7533\u8bf7\u5e74`"}
=== Function Output ===
| 当前申请(专利权)人 | 申请年 | IPC主分类号 |
| --- | --- | --- |
| (株)爱恩德生物 | 2017 | G01N33/68 |
| AMGEN FREMONT INC. | 2002 | A01K |
| GTC生物治疗学公司 | 2000 | C07K1/00 |
| GTC生物治疗学公司 | 2003 | A01K67/027 |
| GTC生物治疗学公司 | 2003 | A01K67/027 |
| GTC生物治疗学公司 | 2004 | C07K1/00 |
| GTC生物治疗学公司 | 2004 | C12N15/00 |
| GTC生物治疗学公司 | 阿斯特拉泽尼卡英国有限公司 | 由卫生与公众服务部代表的美利坚合众国政府 | 2000 | C07H21/04 |
| NPS药物有限公司 | 1998 | G01N33/50 |
| PPL THERAPEUTICS ( SCOTLAND ) LIMITED | 1998 | C12N5/10 |
| 三菱重工业株式会社 | 2010 | A01K67/00 |
| 上海中医药大学 | 2014 | A61K4